# Job Sequencing example with the `qubovert.problems` library

*qubovert* must be pip installed.

Import the Job Sequencing problem from the `qubovert.problems` module.

In [2]:
from qubovert.problems import JobSequencing

Let's create a Job Sequencing problem. We have three jobs that each take a certain amount of time.

- Job 1 takes 2 timesteps.
- Job 2 takes 3 timesteps.
- Job 3 takes 1 timesteps.

And we have two employees that can cover these jobs.

In [3]:
job_lengths = {"job1": 2, "job2": 3, "job3": 1}
num_workers = 2

The goal is to minimize the amount of time the worker who works the longest works, subject to the constraint that each job is covered exactly once.

In [4]:
problem = JobSequencing(job_lengths, num_workers)

To ensure everything is working, let's solve this problem bruteforce.

In [5]:
solutions = problem.solve_bruteforce(all_solutions=True)
print(solutions)

[({'job2'}, {'job3', 'job1'}), ({'job3', 'job1'}, {'job2'})]


We see there are two possible solutions, where one worker gets jobs 1 and 3 and the other worker gets job 2. This means that the worker who works the longest works 3 timesteps, which is indeed the minimum.

Let's convert this problem to a QUBO and solve it with D'Wave's simulated annealer.

In [6]:
#!pip install dwave-neal
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

Note that their software package takes in a specific form for QUBOs, namely, the keys of the dictionary must be two element tuples. This form can be accessed from `Q` with `Q.Q`.

In [7]:
# to_qubo takes lagrange multipliers as option arguments. see the docstring for info.
Q = problem.to_qubo()

# or problem.num_binary_variables
print("Number of qubo variables:", Q.num_binary_variables, "\n")

qubo_sample = sampler.sample_qubo(Q.Q, num_reads=500)
print("objective function:", qubo_sample.first.energy + Q.offset, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution, "\n")

solution = problem.convert_solution(qubo_solution)
print("problem solution:", solution, "\n")

print("The solution is", "valid" if problem.is_solution_valid(solution) else "invalid")

Number of qubo variables: 10 

objective function: 3.0 

qubo solution: {0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0} 

problem solution: ({'job2'}, {'job3', 'job1'}) 

The solution is valid


Notice that the solution is valid because each job is covered exactly once. Also notice that the objective function is equal to `max(sum(job_lengths[i] for i in x) for x in solution)`, ie the duration of the longest workers schedule.

Now let's solve the QUSO formulation to confirm we get the same thing.

In [8]:
# to_quso takes lagrange multipliers as option arguments. see the docstring for info.
L = problem.to_quso()

# or problem.num_binary_variables
print("Number of quso variables:", L.num_binary_variables, "\n")

quso_sample = sampler.sample_ising(L.h, L.J, num_reads=500)
print("objective function:", quso_sample.first.energy + L.offset, "\n")

quso_solution = quso_sample.first.sample
print("quso solution:", quso_solution, "\n")

solution = problem.convert_solution(quso_solution)
print("problem solution:", solution, "\n")

print("The solution is", "valid" if problem.is_solution_valid(solution) else "invalid")

Number of quso variables: 10 

objective function: 3.0 

quso solution: {0: 1, 1: -1, 2: -1, 3: 1, 4: 1, 5: -1, 6: 1, 7: 1, 8: 1, 9: 1} 

problem solution: ({'job2'}, {'job3', 'job1'}) 

The solution is valid


Indeed we get the same result!